# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770794296182                   -0.53    9.0    237ms
  2   -2.772136383783       -2.87       -1.30    1.0    487ms
  3   -2.772169439575       -4.48       -2.63    1.0    109ms
  4   -2.772170720646       -5.89       -4.08    2.0    463ms
  5   -2.772170722929       -8.64       -4.60    2.0    130ms
  6   -2.772170723011      -10.09       -5.51    1.0    114ms
  7   -2.772170723015      -11.37       -5.94    2.0    135ms
  8   -2.772170723015      -12.87       -6.74    1.0    120ms
  9   -2.772170723015      -13.52       -7.72    2.0    149ms
 10   -2.772170723015   +  -13.68       -8.07    2.0    148ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770810670098                   -0.52    9.0    193ms
  2   -2.772061937705       -2.90       -1.32    1.0    108ms
  3   -2.7

1.773557997884673

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770828503671                   -0.52    9.0    161ms
  2   -2.772062033284       -2.91       -1.32    1.0    120ms
  3   -2.772083070424       -4.68       -2.45    1.0    123ms
  4   -2.772083338436       -6.57       -3.14    1.0    126ms
  5   -2.772083416847       -7.11       -4.03    2.0    130ms
  6   -2.772083417694       -9.07       -4.63    1.0    112ms
  7   -2.772083417809       -9.94       -5.33    2.0    132ms
  8   -2.772083417810      -12.14       -5.80    1.0    119ms
  9   -2.772083417811      -12.36       -7.55    1.0    133ms
 10   -2.772083417811      -13.71       -7.95    3.0    167ms
 11   -2.772083417811   +  -13.89       -8.35    1.0    122ms

Polarizability via ForwardDiff:       1.7725349839101703
Polarizability via finite difference: 1.773557997884673
